In [1]:
import numpy as np
import utilitis_FEEC.bsplines as bsp
import scipy.sparse as sparse
import scipy.sparse.linalg as lin
import sympy as sym

import utilitis_FEEC.mass_matrices as ma

#import mass_matrices_stencil as ma_sten


import psydac as ps


from psydac.linalg.iterative_solvers import cg
from psydac.fem.basic                import FemField
from psydac.linalg.stencil           import StencilMatrix, StencilVector, StencilVectorSpace
from psydac.linalg.block             import ProductSpace, BlockVector, BlockLinearOperator, BlockMatrix

In [2]:
p    = 3
bc   = True
Nel  = 5
el_b = np.linspace(0., 1., Nel + 1)
T    = bsp.make_knots(el_b, p, bc)

In [3]:
def mass_1d(T, p, bc):
    
    Nbase            = Nel + p - bc*p
    el_b             = bsp.breakpoints(T, p)
    pts_loc, wts_loc = np.polynomial.legendre.leggauss(p + 1)
    pts,     wts     = bsp.quadrature_grid(el_b, pts_loc, wts_loc)

    basis            = bsp.basis_ders_on_quad_grid(T, p, pts, 0)

    M = np.zeros((Nbase, 2*p + 1))

    for ie in range(Nel):

        for il in range(p + 1):
            for jl in range(p + 1):

                value = 0.

                for q in range(p + 1):
                    value += wts[ie, q] * basis[ie, il, 0, q] * basis[ie, jl, 0, q]

                M[(ie + il)%Nbase, p + jl - il] += value
                
    indices = np.indices((Nbase, 2*p + 1))
    shift   = np.arange(Nbase) - p
    
    row = indices[0].flatten()
    col = (indices[1] + shift[:, None])%Nbase
    
    M = sparse.csr_matrix((M.flatten(), (row, col.flatten())), shape=(Nbase, Nbase))
    M.eliminate_zeros()
                
    return M    

In [4]:
def mass_1d_DN(T, p, bc):
    
    t                = T[1:-1]
    
    NbaseN            = Nel + p - bc*p
    NbaseD            = NbaseN - (1 - bc)
    
    el_b             = bsp.breakpoints(T, p)
    pts_loc, wts_loc = np.polynomial.legendre.leggauss(p + 1)
    pts,     wts     = bsp.quadrature_grid(el_b, pts_loc, wts_loc)

    basisN           = bsp.basis_ders_on_quad_grid(T, p, pts, 0)
    basisD           = bsp.basis_ders_on_quad_grid(t, p - 1, pts, 0, normalize=True)
    
    M = np.zeros((NbaseD, 2*p + 1))

    for ie in range(Nel):

        for il in range(p):
            for jl in range(p + 1):

                value = 0.

                for q in range(p + 1):
                    value += wts[ie, q] * basisD[ie, il, 0, q] * basisN[ie, jl, 0, q]

                M[(ie + il)%NbaseD, p + jl - il] += value
                
    indices = np.indices((NbaseD, 2*p + 1))
    shift   = np.arange(NbaseD) - p

    row = indices[0].flatten()
    col = (indices[1] + shift[:, None])%NbaseN
    
    M = sparse.coo_matrix((M.flatten(), (row, col.flatten())), shape=(NbaseD, NbaseN))
    M.eliminate_zeros()
                
    return M

In [5]:
def mass_1d_ND(T, p, bc):
    
    t                = T[1:-1]
    
    NbaseN            = Nel + p - bc*p
    NbaseD            = NbaseN - (1 - bc)
    
    el_b             = bsp.breakpoints(T, p)
    pts_loc, wts_loc = np.polynomial.legendre.leggauss(p + 1)
    pts,     wts     = bsp.quadrature_grid(el_b, pts_loc, wts_loc)

    basisN           = bsp.basis_ders_on_quad_grid(T, p, pts, 0)
    basisD           = bsp.basis_ders_on_quad_grid(t, p - 1, pts, 0, normalize=True)
    
    M = np.zeros((NbaseN, 2*p + 1))

    for ie in range(Nel):

        for il in range(p + 1):
            for jl in range(p):

                value = 0.

                for q in range(p + 1):
                    value += wts[ie, q] * basisN[ie, il, 0, q] * basisD[ie, jl, 0, q]

                M[(ie + il)%NbaseN, p + jl - il] += value
                
    
    indices = np.indices((NbaseN, 2*p + 1))
    shift   = np.arange(NbaseN) - p

    row = indices[0].flatten()
    col = (indices[1] + shift[:, None])%NbaseD
    
    M = sparse.coo_matrix((M.flatten(), (row, col.flatten())), shape=(NbaseN, NbaseD))
    M.eliminate_zeros()
                
    return M

In [6]:
def mass_1d_DD(T, p, bc):
    
    t                 = T[1:-1]
    
    NbaseN            = Nel + p - bc*p
    NbaseD            = NbaseN - (1 - bc)
    
    el_b             = bsp.breakpoints(T, p)
    pts_loc, wts_loc = np.polynomial.legendre.leggauss(p + 1)
    pts,     wts     = bsp.quadrature_grid(el_b, pts_loc, wts_loc)

    basisN           = bsp.basis_ders_on_quad_grid(T, p, pts, 0)
    basisD           = bsp.basis_ders_on_quad_grid(t, p - 1, pts, 0, normalize=True)
    
    M = np.zeros((NbaseD, 2*p + 1))

    for ie in range(Nel):

        for il in range(p):
            for jl in range(p):

                value = 0.

                for q in range(p + 1):
                    value += wts[ie, q] * basisD[ie, il, 0, q] * basisD[ie, jl, 0, q]

                M[(ie + il)%NbaseD, p + jl - il] += value
                
    
    indices = np.indices((NbaseD, 2*p + 1))
    shift   = np.arange(NbaseD) - p

    row = indices[0].flatten()
    col = (indices[1] + shift[:, None])%NbaseD
    
    M = sparse.csr_matrix((M.flatten(), (row, col.flatten())), shape=(NbaseD, NbaseD))
    M.eliminate_zeros()
                
    return M

In [7]:
p       = [4, 3, 3]
bc      = [False, True, True]
Nel     = [8, 7, 6]
el_b    = [np.linspace(0., 1., Nel + 1) for Nel in Nel]
T       = [bsp.make_knots(el_b, p, bc) for el_b, p, bc in zip(el_b, p, bc)]
Nbase   = [Nel + p - bc*p for Nel, p, bc in zip(Nel, p, bc)]

mapping = 'torus'


if mapping == 'cylinder':
    
    # ... coordinates
    r, phi, z = sym.symbols('r, phi, z')
    q = sym.Matrix([r, phi, z])
    
    # ... mapping
    R1 = 0.2         # inner radius
    R2 = 1.0         # outer radius
    dR = R2 - R1     # thickness
    Lz = 1.          # length
    
    F = sym.Matrix([(r*dR + R1)*sym.cos(2*sym.pi*phi), (r*dR + R1)*sym.sin(2*sym.pi*phi), Lz*z])
    
    xc = sym.lambdify(q, F[0])
    yc = sym.lambdify(q, F[1])
    zc = sym.lambdify(q, F[2])
    
    # ... jacobian matrix
    DF = [[lambda r, phi, z : dR*np.cos(2*np.pi*phi), lambda r, phi, z : -2*np.pi*(r*dR + R1)*np.sin(2*np.pi*phi), lambda r, phi, z : 0*r*phi*z], [lambda r, phi, z : dR*np.sin(2*np.pi*phi), lambda r, phi, z : 2*np.pi*(r*dR + R1)*np.cos(2*np.pi*phi), lambda r, phi, z : 0*r*phi*z], [lambda r, phi, z : 0*r*phi*z, lambda r, phi, z : 0*r*phi*z, lambda r, phi, z : Lz*np.ones(r.shape)]]
    
    # ... metric tensor and its inverse
    G    = [[lambda r, phi, z : dR**2*np.ones(r.shape), lambda r, phi, z : 0*r*phi*z, lambda r, phi, z : 0*r*phi*z], [lambda r, phi, z : 0*r*phi*z, lambda r, phi, z : (2*np.pi)**2*(r*dR + R1)**2, lambda r, phi, z : 0*r*phi*z], [lambda r, phi, z : 0*r*phi*z, lambda r, phi, z : 0*r*phi*z, lambda r, phi, z : Lz**2*np.ones(r.shape)]]
    Ginv = [[lambda r, phi, z : 1/dR**2*np.ones(r.shape), lambda r, phi, z : 0*r*phi*z, lambda r, phi, z : 0*r*phi*z], [lambda r, phi, z : 0*r*phi*z, lambda r, phi, z : 1/((2*np.pi)**2*(r*dR + R1)**2), lambda r, phi, z : 0*r*phi*z], [lambda r, phi, z : 0*r*phi*z, lambda r, phi, z : 0*r*phi*z, lambda r, phi, z : 1/Lz**2*np.ones(r.shape)]]
    
    # ... square root of jacobi determinant
    g      = lambda r, phi, z : dR**2*Lz**2*(2*np.pi)**2*(r*dR + R1)**2
    g_sqrt = lambda r, phi, z : dR*Lz*2*np.pi*(r*dR + R1)
    
    
elif mapping == 'torus':
    
    # ... coordinates
    r, theta, phi = sym.symbols('r, theta, phi')
    q = sym.Matrix([r, theta, phi])

    # ... mapping
    R0 = 1.5         # major radius
    R1 = 0.2         # inner radius
    R2 = 1.0         # outer radius
    dR = R2 - R1     # thickness
    
    F = sym.Matrix([(R0 + (r*dR + R1)*sym.cos(2*sym.pi*theta))*sym.cos(2*sym.pi*phi), (R0 + (r*dR + R1)*sym.cos(2*sym.pi*theta))*sym.sin(2*sym.pi*phi), (r*dR + R1)*sym.sin(2*sym.pi*theta)])
    
    xc = sym.lambdify(q, F[0])
    yc = sym.lambdify(q, F[1])
    zc = sym.lambdify(q, F[2]) 
    
    # ... jacobian matrix
    DF = F.jacobian(q)

    # ... metric tensor and its inverse
    G = sym.simplify(DF.transpose()*DF)
    Ginv = G.inverse()

    # ... square root of jacobi determinant
    g = sym.simplify(G.det())
    g_sqrt = sym.sqrt(g)

    G = [[sym.lambdify(q, G[0, 0]), sym.lambdify(q, G[0, 1]), sym.lambdify(q, G[0, 2])], [sym.lambdify(q, G[1, 0]), sym.lambdify(q, G[1, 1]), sym.lambdify(q, G[1, 2])], [sym.lambdify(q, G[2, 0]), sym.lambdify(q, G[2, 1]), sym.lambdify(q, G[2, 2])]]
    Ginv = [[sym.lambdify(q, Ginv[0, 0]), sym.lambdify(q, Ginv[0, 1]), sym.lambdify(q, Ginv[0, 2])], [sym.lambdify(q, Ginv[1, 0]), sym.lambdify(q, Ginv[1, 1]), sym.lambdify(q, Ginv[1, 2])], [sym.lambdify(q, Ginv[2, 0]), sym.lambdify(q, Ginv[2, 1]), sym.lambdify(q, Ginv[2, 2])]]
    g = sym.lambdify(q, g)
    g_sqrt = sym.lambdify(q, g_sqrt)
    
    
elif mapping == 'cube':
    
    # ... coordinates
    x, y, z = sym.symbols('x, y, z')
    q = sym.Matrix([x, y, z])
    
    # ... mapping
    Lx = 1.           # length in x
    Ly = 2.           # length in y
    Lz = 3.           # length in z
    
    F = sym.Matrix([Lx*x, Ly*y, Lz*z])
    
    xc = sym.lambdify(q, F[0])
    yc = sym.lambdify(q, F[1])
    zc = sym.lambdify(q, F[2])
    
    #... jacobian matrix and its inverse
    DF = [[lambda x, y, z : Lx*np.ones(x.shape), lambda x, y, z : 0*x*y*z, lambda x, y, z : 0*x*y*z], [lambda x, y, z : 0*x*y*z, lambda x, y, z: Ly*np.ones(x.shape), lambda x, y, z : 0*x*y*z], [lambda x, y, z : 0*x*y*z, lambda x, y, z : 0*x*y*z, lambda x, y, z : Lz*np.ones(x.shape)]]
    
    # ... metric tensor and its inverse
    G    = [[lambda x, y, z : Lx**2*np.ones(x.shape), lambda x, y, z : 0*x*y*z, lambda x, y, z : 0*x*y*z], [lambda x, y, z : 0*x*y*z, lambda x, y, z: Ly**2*np.ones(x.shape), lambda x, y, z : 0*x*y*z], [lambda x, y, z : 0*x*y*z, lambda x, y, z : 0*x*y*z, lambda x, y, z : Lz**2*np.ones(x.shape)]]
    Ginv = [[lambda x, y, z : 1/Lx**2*np.ones(x.shape), lambda x, y, z : 0*x*y*z, lambda x, y, z : 0*x*y*z], [lambda x, y, z : 0*x*y*z, lambda x, y, z: 1/Ly**2*np.ones(x.shape), lambda x, y, z : 0*x*y*z], [lambda x, y, z : 0*x*y*z, lambda x, y, z : 0*x*y*z, lambda x, y, z : 1/Lz**2*np.ones(x.shape)]]
    
    #... square root of jacobi determinant
    g      = lambda x, y, z : Lx**2*Ly**2*Lz**2*np.ones(x.shape)
    g_sqrt = lambda x, y, z : Lx*Ly*Lz*np.ones(x.shape)

In [8]:
# Create 1D finite element spaces
V1 = ps.fem.splines.SplineSpace(p[0], grid=el_b[0], periodic=False)
V2 = ps.fem.splines.SplineSpace(p[1], grid=el_b[1], periodic=True)
V3 = ps.fem.splines.SplineSpace(p[2], grid=el_b[2], periodic=True)

# Create 3D tensor product finite element space
V = ps.fem.tensor.TensorFemSpace(V1, V2, V3)

In [9]:
M0 = ma.mass_V0(T, p, bc, g_sqrt)

In [10]:
M0

<504x504 sparse matrix of type '<class 'numpy.float64'>'
	with 155232 stored elements in Compressed Sparse Row format>

In [55]:
M0_sten = ma_sten.mass_matrix_V0(V, p, [Nel[0] + p[0], Nel[1] + p[1], Nel[2] + p[2]], T, g_sqrt, bc)

In [56]:
np.allclose(M0_sten.toarray(), M0.toarray())

True

In [57]:
fun = lambda r, phi, z : r*np.sin(2*np.pi*r)*np.sin(4*np.pi*phi)

In [58]:
F0 = ma.inner_prod_V0(T, p, bc, g_sqrt, fun)

In [59]:
F0.shape

(12, 7, 6)

In [60]:
F0_sten = ma_sten.inner_prod_V0(V, fun, p, [Nel[0] + p[0], Nel[1] + p[1], Nel[2] + p[2]], T, g_sqrt, bc)

In [61]:
np.allclose(F0.flatten(), F0_sten.toarray())

True

In [62]:
coeff, info = lin.cg(M0, F0.flatten(), tol=1e-10) 

In [63]:
coeff_sten, info = cg(M0_sten, F0_sten, tol=1e-10)

In [64]:
np.allclose(coeff.flatten(), coeff_sten.toarray())

True

In [66]:
coeff = np.reshape(coeff, (12, 7, 6))

In [67]:
error = ma.L2_error_V0(coeff, T, p, bc, g_sqrt, fun)

In [68]:
error

0.04381653304415181

In [69]:
# ... create Fem field
fun_h = FemField(V, coeffs=coeff_sten)
fun_h.coeffs.update_ghost_regions()

integrand = lambda *x : (fun(*x) - fun_h(*x))**2*g_sqrt(*x) 

In [70]:
error_sten = np.sqrt(V.integral(integrand))

In [71]:
error_sten

0.043816533044151465

In [72]:
M3 = ma.mass_V3(T, p, bc, g_sqrt)

In [73]:
M3

<462x462 sparse matrix of type '<class 'numpy.float64'>'
	with 68250 stored elements in Compressed Sparse Row format>

In [74]:
M3_sten = ma_sten.mass_matrix_V3(p, [Nel[0] + p[0], Nel[1] + p[1], Nel[2] + p[2]], T, g_sqrt, bc)

In [75]:
np.allclose(M3_sten.toarray(), M3.toarray())

True

In [76]:
M1 = ma.mass_V1(T, p, bc, Ginv, g_sqrt)

In [77]:
M1_sten, V1 = ma_sten.mass_matrix_V1(p, [Nel[0] + p[0], Nel[1] + p[1], Nel[2] + p[2]], T, Ginv, g_sqrt, bc)

In [78]:
from psydac.linalg.block import BlockMatrix

M1_sten = BlockMatrix(V1, V1, blocks=M1_sten).tosparse()

In [79]:
np.allclose(M1_sten.toarray(), M1.toarray())

True

In [80]:
M2 = ma.mass_V2(T, p, bc, G, g_sqrt)

In [81]:
M2_sten, V2 = ma_sten.mass_matrix_V2(p, [Nel[0] + p[0], Nel[1] + p[1], Nel[2] + p[2]], T, G, g_sqrt, bc)

In [82]:
from psydac.linalg.block import BlockMatrix

M2_sten = BlockMatrix(V2, V2, blocks=M2_sten).tosparse()

In [83]:
np.allclose(M2_sten.toarray(), M2.toarray())

True

In [84]:
fun = [lambda r, phi, z : r*np.sin(2*np.pi*r)*np.sin(4*np.pi*phi), lambda r, phi, z : r**2*np.sin(2*np.pi*r)*np.sin(4*np.pi*phi), lambda r, phi, z : r*np.sin(2*np.pi*r)*np.sin(4*np.pi*phi), lambda r, phi, z : r*np.sin(2*np.pi*r)*np.cos(4*np.pi*phi)]

In [85]:
F1 = ma.inner_prod_V1(T, p, bc, Ginv, g_sqrt, fun)

In [86]:
F1 = np.concatenate((F1[0].flatten(), F1[1].flatten(), F1[2].flatten()))

In [87]:
F1_sten, V1 = ma_sten.inner_prod_V1(fun, p, [Nel[0] + p[0], Nel[1] + p[1], Nel[2] + p[2]], T, Ginv, g_sqrt, bc)

In [88]:
F1_sten = np.concatenate((F1_sten[0].toarray(), F1_sten[1].toarray(), F1_sten[2].toarray()))

In [89]:
np.allclose(F1, F1_sten)

True

In [90]:
F2 = ma.inner_prod_V2(T, p, bc, G, g_sqrt, fun)

In [91]:
F2 = np.concatenate((F2[0].flatten(), F2[1].flatten(), F2[2].flatten()))

In [92]:
F2.shape

(1428,)

In [93]:
F2_sten, V2 = ma_sten.inner_prod_V2(fun, p, [Nel[0] + p[0], Nel[1] + p[1], Nel[2] + p[2]], T, G, g_sqrt, bc)

In [94]:
F2_sten = np.concatenate((F2_sten[0].toarray(), F2_sten[1].toarray(), F2_sten[2].toarray()))

In [95]:
np.allclose(F2, F2_sten)

True

In [96]:
fun = lambda r, phi, z : r*np.sin(2*np.pi*r)*np.sin(4*np.pi*phi)

In [97]:
F3 = ma.inner_prod_V3(T, p, bc, g_sqrt, fun)

In [98]:
F3.shape

(11, 7, 6)

In [99]:
F3_sten = ma_sten.inner_prod_V3(fun, p, [Nel[0] + p[0], Nel[1] + p[1], Nel[2] + p[2]], T, g_sqrt, bc)

In [100]:
np.allclose(F3.flatten(), F3_sten.toarray())

True